In [1]:
# 模型训练-验证

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [3]:
train_path = r'E:\DataSet\Tianchi\zhengqi\zhengqi_train.txt'
test_path = r'E:\DataSet\Tianchi\zhengqi\zhengqi_test.txt'

train_data = pd.read_csv(train_path, delimiter="\t", encoding='utf-8')
test_data = pd.read_csv(test_path, delimiter="\t", encoding='utf-8')

In [5]:
# 数据归一化处理

from sklearn.preprocessing import MinMaxScaler
feature_columns = [col for col in train_data.columns if col not in  ['target']]
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(train_data[feature_columns])
train_data_scaler = min_max_scaler.transform(train_data[feature_columns])
test_data_scaler = min_max_scaler.transform(test_data[feature_columns])
train_data_scaler = pd.DataFrame(train_data_scaler)
test_data_scaler= pd.DataFrame(test_data_scaler)
train_data_scaler.columns = feature_columns
train_data_scaler['target'] = train_data['target']
test_data_scaler.columns = feature_columns
# display(train_data_scaler.describe())

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V29,V30,V31,V32,V33,V34,V35,V36,V37,target
count,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,...,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000
mean,0.690528,0.735521,0.593745,0.606301,0.639876,0.607801,0.735418,0.741337,0.702012,0.894246,...,0.401715,0.634030,0.760517,0.631794,0.459255,0.484465,0.734850,0.336306,0.527854,0.126353
std,0.143747,0.133738,0.145844,0.151302,0.119550,0.193919,0.141872,0.137111,0.129082,0.067047,...,0.141644,0.125338,0.110903,0.139979,0.099782,0.101353,0.122917,0.123733,0.153423,0.983966
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.044000
25%,0.625465,0.695419,0.497279,0.515165,0.586328,0.497566,0.659249,0.682314,0.653453,0.878743,...,0.300053,0.586993,0.722656,0.565408,0.409037,0.454490,0.684936,0.279760,0.427112,-0.350250
50%,0.727076,0.766264,0.609155,0.609933,0.652940,0.642456,0.767115,0.774045,0.728557,0.909110,...,0.385611,0.633755,0.782330,0.634615,0.454518,0.499949,0.755580,0.349860,0.519532,0.313000
75%,0.783922,0.812642,0.694342,0.714174,0.712185,0.759266,0.835613,0.836958,0.781029,0.909110,...,0.488154,0.694136,0.824949,0.714950,0.504261,0.511365,0.785260,0.414511,0.622210,0.793250
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.538000


In [6]:
# pca 降维，保留16个特征
from sklearn.decomposition import PCA
pca = PCA(n_components=16)
new_train_pca_16 = pca.fit_transform(train_data_scaler.iloc[:, :-1])
new_test_pca_16 = pca.fit_transform(test_data_scaler)
new_train_pca_16 = pd.DataFrame(new_train_pca_16)
new_test_pca_16 = pd.DataFrame(new_test_pca_16)
new_train_pca_16['target'] = train_data_scaler['target']


In [8]:
# 将数据进行切分
from sklearn.model_selection import train_test_split

new_train_pca_16 = new_train_pca_16.fillna(0)
train = new_train_pca_16[new_test_pca_16.columns]
target = new_train_pca_16['target']

# train_test_split?
train_data, test_data, train_target, test_target = train_test_split(train, target, test_size=0.2, random_state=0 )

In [9]:
# 欠拟合
clf = SGDRegressor(max_iter=500, tol=1e-2)
clf.fit(train_data, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data))
score_test = mean_squared_error(test_target, clf.predict(test_data))
print('SGDRegressor train score: ', score_train)
print('SGDRegressor test score: ', score_test)

SGDRegressor train score:  0.15145464727954538
SGDRegressor test score:  0.1557395424638947


In [15]:
# 过拟合
from sklearn.preprocessing import PolynomialFeatures
# PolynomialFeatures?
poly = PolynomialFeatures(degree=5)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.fit_transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print('PolynomialFeatures 5 train score: ', score_train)
print('PolynomialFeatures 5  test score: ', score_test)

PolynomialFeatures 5 train score:  0.13235408041938118
PolynomialFeatures 5  test score:  0.14476785647048604


In [16]:
# 正常拟合
poly = PolynomialFeatures(degree=3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.fit_transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print('PolynomialFeatures 5 train score: ', score_train)
print('PolynomialFeatures 5  test score: ', score_test)

PolynomialFeatures 5 train score:  0.1340909154248941
PolynomialFeatures 5  test score:  0.1425142188842414


In [17]:
# 模型正则化
# L2范数
poly = PolynomialFeatures(degree=3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.fit_transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3, penalty='L2')
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print('L2 train score: ', score_train)
print('L2 test score: ', score_test)

L2 train score:  0.1351862901894467
L2 test score:  0.14339244273112178


In [18]:
# L1范数
poly = PolynomialFeatures(degree=3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.fit_transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3, penalty='L1')
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print('L1 train score: ', score_train)
print('L1 test score: ', score_test)

L2 train score:  0.13520502643322446
L2 test score:  0.14307212748858583


In [22]:
# 弹性网络
poly = PolynomialFeatures(degree=3)
train_data_poly = poly.fit_transform(train_data)
test_data_poly = poly.fit_transform(test_data)
clf = SGDRegressor(max_iter=1000, tol=1e-3, penalty='elasticnet')
clf.fit(train_data_poly, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data_poly))
score_test = mean_squared_error(test_target, clf.predict(test_data_poly))
print('elasticnet train score: ', score_train)
print('elasticnet test score: ', score_test)

L2 train score:  0.1342818629360411
L2 test score:  0.14240448231638014


In [21]:
# 交叉验证
# 简单交叉验证
clf = SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(train_data, train_target)
score_train = mean_squared_error(train_target, clf.predict(train_data))
score_test = mean_squared_error(test_target, clf.predict(test_data))
print('SGDRegressor train score: ', score_train)
print('SGDRegressor test score: ', score_test)

In [30]:
# K折交叉验证
from sklearn.model_selection import KFold
# ValueError: Found input variables with inconsistent numbers of samples: [2311, 2310]

# kf = KFold(n_splits=5)
# for k, (train_index, test_index) in enumerate(kf.split(train)):
#     if (k == 3):
#         print(train.shape)
#         print(train_index)
#         print(test_index)
#     train_data, test_data, train_taret, test_target = train.values[train_index],train.values[test_index],target[train_index],target[test_index]
#     clf = SGDRegressor(max_iter=1000, tol=1e-3)
#     clf.fit(train_data, train_target)
#     score_train = mean_squared_error(train_target, clf.predict(train_data))
#     score_test = mean_squared_error(test_target, clf.predict(test_data))
#     print(k, '折', 'SGDRegressor train score: ', score_train)
#     print(k, '折', 'SGDRegressor test score: ', score_test)

0 折 SGDRegressor train score:  0.9474942105521872
0 折 SGDRegressor test score:  0.9065034130847645
1 折 SGDRegressor train score:  0.9462105912668638
1 折 SGDRegressor test score:  1.1021441017004299
2 折 SGDRegressor train score:  0.9462727757960594
2 折 SGDRegressor test score:  0.925211732395715
(2888, 16)
[   0    1    2 ... 2885 2886 2887]
[1734 1735 1736 1737 1738 1739 1740 1741 1742 1743 1744 1745 1746 1747
 1748 1749 1750 1751 1752 1753 1754 1755 1756 1757 1758 1759 1760 1761
 1762 1763 1764 1765 1766 1767 1768 1769 1770 1771 1772 1773 1774 1775
 1776 1777 1778 1779 1780 1781 1782 1783 1784 1785 1786 1787 1788 1789
 1790 1791 1792 1793 1794 1795 1796 1797 1798 1799 1800 1801 1802 1803
 1804 1805 1806 1807 1808 1809 1810 1811 1812 1813 1814 1815 1816 1817
 1818 1819 1820 1821 1822 1823 1824 1825 1826 1827 1828 1829 1830 1831
 1832 1833 1834 1835 1836 1837 1838 1839 1840 1841 1842 1843 1844 1845
 1846 1847 1848 1849 1850 1851 1852 1853 1854 1855 1856 1857 1858 1859
 1860 1861 1862 18

ValueError: Found input variables with inconsistent numbers of samples: [2311, 2310]

In [27]:
# 留一法验证
from sklearn.model_selection import LeaveOneOut
# loo = LeaveOneOut()
# # kf = KFold(n_splits=5)
# for k, (train_index, test_index) in enumerate(loo.split(train)):
#     train_data, test_data, train_taret, test_target = train.values[train_index],train.values[test_index],target[train_index],target[test_index]
#     clf = SGDRegressor(max_iter=1000, tol=1e-3)
#     clf.fit(train_data, train_target)
#     score_train = mean_squared_error(train_target, clf.predict(train_data))
#     score_test = mean_squared_error(test_target, clf.predict(test_data))
#     print(k, '折', 'SGDRegressor train score: ', score_train)
#     print(k, '折', 'SGDRegressor test score: ', score_test)
#     if k > 9:
#         break

ValueError: Found input variables with inconsistent numbers of samples: [2887, 2310]

In [32]:
# 调参 穷举网格搜索
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(train, target, test_size=0.2, random_state=0)
randomFrestRegression = RandomForestRegressor()
parameters = {'n_estimators': [50, 100, 200], 'max_depth': [1, 2, 3]}
clf = GridSearchCV(randomFrestRegression, parameters, cv=5)
clf.fit(train_data, train_target)
score_test = mean_squared_error(test_target, clf.predict(test_data))
print('RandomForestRegressor GridSearchCV test score', score_test)
sorted(clf.cv_results_.keys())

RandomForestRegressor GridSearchCV test score 0.2559431927020715


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_max_depth',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [33]:
# 调参 随机搜索
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(train, target, test_size=0.2, random_state=0)
randomFrestRegression = RandomForestRegressor()
parameters = {'n_estimators': [50, 100, 200, 300], 'max_depth': [1, 2, 3, 4, 5]}
clf = RandomizedSearchCV(randomFrestRegression, parameters, cv=5)
clf.fit(train_data, train_target)
score_test = mean_squared_error(test_target, clf.predict(test_data))
print('RandomizedSearchCV GridSearchCV test score', score_test)
# sorted(clf.cv_results_.keys())
# print(clf.best_params_)
# {'n_estimators': 50, 'max_depth': 5}

RandomizedSearchCV GridSearchCV test score 0.19878427647218627


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_max_depth',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [39]:
# 调参 随机搜索
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import lightgbm as lgb

train_data, test_data, train_target, test_target = train_test_split(train, target, test_size=0.2, random_state=0)
clf = lgb.LGBMRegressor(num_leaves=31)
# clf?
parameters = {'learning_rate': [0.01, 0.1, 1], 'n_estimators': [20, 40]}
clf = GridSearchCV(clf, parameters, cv=5)
clf.fit(train_data, train_target)
score_test = mean_squared_error(test_target, clf.predict(test_data))
print('LGBMRegressor GridSearchCV test score', score_test)
print('LGBMRegressor best parameters: ', clf.best_params_)

LGBMRegressor GridSearchCV test score 0.15048537417492472
LGBMRegressor best parameters:  {'learning_rate': 0.1, 'n_estimators': 40}
